## 利用torch.autograd进行自动微分

在训练神经网络时，最常用的算法是反向传播。  
该算法中，参数(模型权值)根据损失函数相对于给定参数的梯度进行调整。  

为了计算这些梯度，PyTorch内置了一个名为torch.autograd的差异化引擎。它支持自动计算梯度的任何计算图。  

In [1]:
import torch

# 最简单的单层神经网络模型，输入x，参数w和b，以及一些损失函数
x = torch.ones(5)  # input tensor
y = torch.zeros(3)  # expected output
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
z = torch.matmul(x, w)+b
loss = torch.nn.functional.binary_cross_entropy_with_logits(z, y)

## 张量、函数、计算图
在这个网络中，w和b是需要优化的参数。  
因此，我们需要能够计算损失函数相对于这些变量的梯度。  
为了做到这一点，我们设置这些张量的requires_grad属性。

In [2]:
# 我们应用在张量上构造计算图的函数实际上是一个函数类的对象。
# 该对象知道如何在正向计算函数，也知道如何在反向传播步骤中计算其导数。
# 对向后传播函数的引用存储在张量的grad_fn属性中。
print('Gradient function for z =', z.grad_fn)
print('Gradient function for loss =', loss.grad_fn)

Gradient function for z = <AddBackward0 object at 0x7f8e20638760>
Gradient function for loss = <BinaryCrossEntropyWithLogitsBackward object at 0x7f8e20638820>


## 计算梯度

In [3]:
loss.backward()
print(w.grad)
print(b.grad)

tensor([[0.1721, 0.0132, 0.2082],
        [0.1721, 0.0132, 0.2082],
        [0.1721, 0.0132, 0.2082],
        [0.1721, 0.0132, 0.2082],
        [0.1721, 0.0132, 0.2082]])
tensor([0.1721, 0.0132, 0.2082])


## disable 梯度反向传播
默认情况下，所有require_grad =True的张量都会跟踪它们的计算历史，并支持梯度计算。  
但是，在某些情况下，我们不需要这样做，例如，我们训练了模型，只是想把它应用到一些输入数据上，即我们只想通过网络进行正向计算。  
我们可以通过使用torch.no_grad()块包围我们的计算代码来停止跟踪计算。

In [4]:
z = torch.matmul(x, w)+b
print(z.requires_grad)

with torch.no_grad():
    z = torch.matmul(x, w)+b
print(z.requires_grad)

True
False


In [5]:
# 实现相同结果的另一种方法是在张量上使用 detach() 方法：
z = torch.matmul(x, w)+b
z_det = z.detach()
print(z_det.requires_grad)
"""
    以下是禁用梯度跟踪的原因:
        将神经网络中的一些参数标记为冻结参数。这是对预先训练的网络进行微调的一个非常常见的场景
        在只进行正向传递的情况下加快计算速度，因为在不跟踪梯度的张量上的计算将更加有效。
"""

False


## 计算图补充
从概念说，autograd操作在一个由Function对象组成的有向无环图（DAG）中保存一组数据（张量）和所有执行的操作（以及产生的新张量）。  
在这个DAG中，输入张量可以视作叶，输出张量视作根。通过从根到叶跟踪这个图，可以使用链式法则自动计算梯度。  

在向前传播时，autograd会同时做两件事:  
1、运行请求的操作来计算结果张量  
2、在DAG中保持操作的梯度函数  

当在DAG根目录上调用.backward()时，反向传播开始。autograd:  
1、计算每个梯度  
2、将它们累加到各自张量的.grad属性中  
4、利用链式法则，一直传播到叶张量  

在PyTorch中，DAGs是动态的。如果需要，可以在每次迭代中更改形状、大小和操作。